In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import datetime
from pprint import pprint as pp

In [10]:
def generate_random_sparse_mask(size_in, size_out, sparsity):
    """ make sure each node has at least one edge connecting to it"""
    while True:
        try:
            a = np.random.uniform(low=0, high=1, size=(size_in, size_out))
            a = (a>(1-sparsity)).astype(int)
            assert((a.sum(axis=1)>1).all())
            assert((a.sum(axis=0)>1).all())
            break
        except AssertionError:
            continue
    return tf.constant(a.astype('float32'))

In [30]:
size = (5, 3)
lr = 100
tf.reset_default_graph()
mask = generate_random_sparse_mask(size[0], size[1], 0.5)
y_dense = tf.Variable(tf.constant(np.ones(size).astype("float32")))
y_sparse = tf.Variable(tf.constant(np.ones(size).astype("float32")))
y_target = tf.constant(2*np.ones(size).astype("float32"))
loss_dense = tf.reduce_mean(tf.square(y_dense-y_target))
step_dense = tf.train.AdadeltaOptimizer(lr).minimize(loss_dense)
loss_sparse = tf.reduce_mean(tf.square(tf.multiply(mask, y_sparse)-tf.multiply(mask, y_target)))
step_sparse = tf.train.AdadeltaOptimizer(lr).minimize(loss_sparse)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        sess.run([step_sparse, step_dense])
    y_d, y_s = sess.run([y_dense, y_sparse])
    mask = sess.run(mask)

In [31]:
y_d

array([[1.999631, 1.999631, 1.999631],
       [1.999631, 1.999631, 1.999631],
       [1.999631, 1.999631, 1.999631],
       [1.999631, 1.999631, 1.999631],
       [1.999631, 1.999631, 1.999631]], dtype=float32)

In [32]:
y_s

array([[1.      , 1.999631, 1.999631],
       [1.999631, 1.      , 1.999631],
       [1.999631, 1.999631, 1.      ],
       [1.999631, 1.999631, 1.999631],
       [1.999631, 1.999631, 1.      ]], dtype=float32)

In [33]:
mask

array([[0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 0.],
       [1., 1., 1.],
       [1., 1., 0.]], dtype=float32)